# Generate Data For Analysis of Glint Visibility

This notebook generates data for analysis of glint visibility.
Generation of 100 combined user-device samples x two eyes x 20 gaze directions,
i.e., 4,000 individual samples, takes about 25-40 min on a beefy machine.

In [1]:
%matplotlib widget

import datetime
import kiruna
import os
import pickle
import sys

sys.path.append("..")
import utils

dropdown_widget, text_widget = utils.get_experiment_info()

RadioButtons(description='Device:', index=4, options=('atlas_1.2', 'palau', 'sydney', 'tasman', 'p47', 'p47_PO…

Text(value='.\\results\\p47', description='Results:', placeholder="Default is '.'")

## Data Generation

In [2]:
scene_file_name, \
    sampler_file_name = \
    utils.get_configuration_files(dropdown_widget.value)

# print("Scene generated using " + scene_file_name + " configuration file.")
et_scene = kiruna.scene.SceneModel(parameter_file_name=scene_file_name)

print("Sampling parameters from " + sampler_file_name + " configuration file.")
scene_sampler = kiruna.sampler.SceneSampler(
    et_scene, num_samples=100, parameter_file_name=sampler_file_name)

df = scene_sampler.generate_data_for_visibility_analysis()

now = datetime.datetime.now()
prefix = now.strftime("%Y-%m-%d @ %H-%M-%S.%f")
results_path = text_widget.value
os.makedirs(results_path, exist_ok=True)
path_prefix = os.path.join(results_path, prefix)

df_name = path_prefix + " data_frame.pkl"
with open(df_name, 'wb') as file_stream:
    pickle.dump(df, file_stream)

df

Sampling parameters from C:\Users\jayanez\Documents\mixedreality.Platform.EyeTracking.Tools\kiruna\kiruna\sampler\default_sampler/default_scene_sampler.json configuration file.


,Subsystem,Horiz. angle,Vert. angle,LED 01,LED 02,LED 03,LED 04,LED 05,LED 06,LED 07,...,LED 12,Camera center in pupil x,Camera center in pupil y,Camera center in pupil z,Delta eye relief,Scene index,Gaze x,Gaze y,Gaze z,IPD
0,0,-20.0,-15.0,1,0,0,0,0,0,0,...,1,-5.963615,-31.503361,10.831165,1.298601,0,-0.334993,0.130487,0.933141,59.872074
1,0,-20.0,-5.0,1,1,0,0,0,0,0,...,1,-4.061184,-27.877722,15.756128,0.903169,0,-0.334993,-0.033534,0.941624,59.815605
2,0,-20.0,5.0,1,1,0,0,0,0,0,...,1,-2.365273,-23.391394,20.000708,0.920092,0,-0.334993,-0.196535,0.921495,59.811607
3,0,-20.0,15.0,1,1,0,0,0,0,0,...,1,-0.927401,-18.180683,23.435934,1.348854,0,-0.334993,-0.353566,0.873368,59.86021
4,0,-10.0,-15.0,1,0,0,0,0,0,0,...,1,-8.920647,-31.653772,9.537602,0.695869,0,-0.167522,0.141866,0.975608,61.631348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,1,10.0,15.0,1,1,0,0,0,0,1,...,1,5.586543,-20.017820,25.457655,0.708066,99,0.164222,-0.387557,0.907100,61.789284
3996,1,20.0,-15.0,1,0,0,0,0,0,0,...,1,5.370238,-34.405865,12.133447,1.289230,99,0.331294,0.106696,0.937475,60.01267
3997,1,20.0,-5.0,1,1,0,0,0,0,0,...,1,3.282090,-30.480577,17.518528,0.896319,99,0.331294,-0.057716,0.941760,59.96924
3998,1,20.0,5.0,1,1,0,0,0,0,0,...,1,1.407705,-25.613585,22.162491,0.916370,99,0.331294,-0.220374,0.917431,59.978413
